In [1]:
import sys
import os
import numpy as np
from _3dpu_improved import *
ROOT = "../"

In [2]:
import yaml

# Load the YAML file
with open(ROOT + 'paths.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access paths
data_path = config['paths']["data_big"]




In [3]:
data_path

'/home/mehdii/projects/def-vidalthi/mehdii/3DPU_Plateau_Problem/Data/ph.nii'

In [4]:
import nibabel as nb
time = 3
data = nb.load(data_path).get_fdata()
data = np.array(data)[:,:,:,time]

In [36]:
C = Resiuals(data)

In [41]:
C.map_nodes()
for res in tqdm(C.list_res):
    i,j,k,axe,value = res
    # if i not in [0,X-1] and j not in [0,Y-1] and k not in [0,Z-1]:
    C.nodes_of_not_boundary(res)

  0%|          | 0/6069798 [00:00<?, ?it/s]


KeyError: 4046532

In [45]:
def prepare_mapping_dict():
    residual_to_index = {residual: index for index, residual in enumerate(C.mapping)}
    return residual_to_index
R = prepare_mapping_dict()


In [46]:
from tqdm import tqdm
for r in tqdm(C.list_res):
    C.Res_graph[R[r]] = []

  0%|          | 0/6069798 [00:00<?, ?it/s]

100%|██████████| 6069798/6069798 [00:09<00:00, 632648.60it/s] 


In [44]:
R

[0]

In [39]:
for res in tqdm(C.list_res):
    i,j,k,axe,value = res
    # if i not in [0,X-1] and j not in [0,Y-1] and k not in [0,Z-1]:
    C.nodes_of_not_boundary(res)

  0%|          | 0/2023266 [00:00<?, ?it/s]


KeyError: 0

In [32]:
C.identify_connected_component()

In [34]:
D = (C.connected_components).values()
Number = [len(k) for k in D]
R = sorted(Number)
R

[0]

In [27]:
R

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [22]:
Number

[2,
 1781553,
 3,
 5,
 6,
 2,
 2,
 2,
 5,
 2,
 2,
 5,
 1,
 5,
 5,
 1,
 4,
 2,
 8,
 4,
 2,
 6,
 2,
 4,
 10,
 4,
 4,
 4,
 4,
 16,
 6,
 4,
 6,
 15,
 8,
 4,
 4,
 8,
 4,
 4,
 12,
 4,
 2,
 3,
 6,
 24,
 4,
 12,
 6,
 4,
 4,
 2,
 4,
 4,
 6,
 4,
 4,
 8,
 8,
 22,
 4,
 14,
 8,
 6,
 2,
 4,
 6,
 2,
 10,
 4,
 4,
 4,
 2,
 7,
 4,
 8,
 10,
 14,
 4,
 4,
 6,
 6,
 6,
 4,
 9,
 4,
 3,
 8,
 4,
 2,
 12,
 6,
 2,
 2,
 4,
 6,
 10,
 4,
 4,
 9,
 6,
 4,
 8,
 4,
 4,
 10,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 1,
 6,
 4,
 2,
 10,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 4,
 14,
 6,
 4,
 1,
 4,
 4,
 6,
 3,
 4,
 4,
 6,
 4,
 4,
 6,
 10,
 5,
 4,
 4,
 14,
 4,
 4,
 10,
 6,
 12,
 8,
 4,
 16,
 2,
 8,
 3,
 6,
 4,
 4,
 4,
 4,
 14,
 12,
 6,
 4,
 4,
 4,
 4,
 4,
 10,
 9,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 10,
 4,
 4,
 4,
 5,
 6,
 10,
 6,
 5,
 4,
 6,
 6,
 4,
 4,
 8,
 11,
 4,
 11,
 2,
 4,
 14,
 3,
 4,
 2,
 3,
 1,
 2,
 1,
 1,
 3,
 5,
 8,
 4,
 3,
 5,
 2,
 3,
 4,
 7,
 2,
 2,
 23,
 2,
 3,
 7,
 2,
 2,
 16,
 3,
 3,
 3,
 4,
 2,
 2,
 6,
 2,
 19,
 3

In [13]:
0 in (set(list(C.connex.values())))

False

In [14]:
R = []
for key in tqdm(C.Res_graph.keys()):
    if len(C.Res_graph[key]) not in [0,1,2,3]:
        R.append(key)

100%|██████████| 2023266/2023266 [00:00<00:00, 2402570.20it/s]
